## Bitcoin Transaction Retrieval using Google BigQuery

In [ ]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd

In [ ]:
auth.authenticate_user() # connects to your Google account to access BigQuery

In [ ]:
project_id = 'clever-arcadia-342004'
client = bigquery.Client(project=project_id)
dataset_ref = client.dataset('crypto_bitcoin', project='bigquery-public-data')

- crypto_bitcoin_cash, crypto_ethereum, crypto_ethereum_classic, crypto_dash, crypto_dogecoin, crypto_litecoin, crypto_zcash are also available


In [ ]:
dataset = client.get_dataset(dataset_ref)
table_ref = dataset.table('transactions')
table = client.get_table(table_ref)

In [ ]:
# query the first 10 transactions made on the Bitcoin blockchain
N = 10 * 2
# multipled by 2 because BTC transactions are first sending the total amount held by the sender 
# and then the sender receives the change back in another transaction

query = """
    SELECT block_timestamp, inputs.addresses, outputs.addresses, outputs.value, fee
    FROM `bigquery-public-data.crypto_bitcoin.transactions` AS transactions, transactions.outputs as outputs, transactions.inputs as inputs
    WHERE transactions.block_timestamp_month = "2009-01-01"
    LIMIT """ + str(N) 
   
query_job = client.query(query) 

In [ ]:
dates = list()
senders = list()
receivers = list()
amounts = list()
fees = list()

i = 0
for row in query_job:
  if (row[1][0]!=row[2][0]):
    dates.append(row[0])
    senders.append(row[1][0])
    receivers.append(row[2][0])
    amounts.append(row[3])
    fees.append(row[4])
  i+=1
    

In [ ]:
df = pd.DataFrame({'Date': dates, 'Sender': senders, 'Receiver': receivers, 'Amount': amounts, 'Fees':fees})

In [ ]:
df

,Date,Sender,Receiver,Amount,Fees
0,2009-01-12 06:02:13+00:00,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1DUDsfc23Dv9sPMEk5RsrtfzCw5ofi5sVW,1000000000,0
1,2009-01-12 06:12:16+00:00,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1LzBzVqEeuQyjD2mRWHes3dgWrT9titxvq,100000000,0
2,2009-01-12 06:34:22+00:00,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,13HtsYzne8xVPdGDnmJX8gHgBZerAfJGEf,100000000,0
3,2009-01-12 20:04:20+00:00,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1ByLSV2gLRcuqUmfdYcpPQH8Npm8cccsFg,1000000000,0
4,2009-01-15 05:48:32+00:00,1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi,1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5,100000000,0
5,2009-01-15 06:08:20+00:00,1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5,1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv,100000000,0
6,2009-01-15 06:08:20+00:00,1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi,1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv,100000000,0
7,2009-01-15 06:08:20+00:00,1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi,1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5,2400000000,0
8,2009-01-15 06:08:20+00:00,1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5,1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv,100000000,0
9,2009-01-15 06:08:20+00:00,1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5,1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv,2400000000,0
